In [1]:
from contextlib import contextmanager
import sqlite3
import yaml

In [2]:
! pwd

/home/jeremy/projects/hyperborea3/scripts/spells


In [3]:
load_folder = "lvl2"

In [4]:
@contextmanager
def db_cur():
    URI = f"../../hyperborea3/hyperborea.sqlite3"
    con = sqlite3.connect(URI, check_same_thread=False)
    con.row_factory = sqlite3.Row
    cur = con.cursor()
    yield cur
    con.commit()
    con.close()

Test connection to db

In [5]:
with db_cur() as cur:
    cur.execute("""
        SELECT *
          FROM spells;
    """)
    result = [dict(x) for x in cur.fetchall()]
result[:1]

[{'spell_id': 1,
  'spell_name': 'Acid Arrow',
  'rng': '30 feet',
  'dur': 'special',
  'reversible': 0,
  'pp': 175,
  'spell_desc': '<p>A magic arrow darts from the finger of the caster. On a successful attack roll (<i>dexterity</i> modifier applies), the acid arrow strikes for 1d4+1 hp physical damage, plus an additional 2d4 hp acid damage in the same round. Magicians (but not other sorcerers) enjoy a +1 bonus to the attack roll for every 2 CA levels (CA 3–4 = +2, CA 5–6 = +3, and so forth). Acid damage will persist for higher level sorcerers:</p> <ul> <li>1 extra round for CA 4–6</li> <li>2 extra rounds for CA 7–9</li> <li>3 extra rounds for CA 10 or greater.</li> </ul> <p>For example, an <i>acid arrow</i> fired by a CA 12 sorcerer on round 1 would inflict 1d4+1 hp base damage plus 2d4 hp acid damage on round 1, 2d4 hp acid damage on round 2, 2d4 hp acid damage on round 3, and a final 2d4 hp acid damage on round 4. The acid may ruin armour or clothing per referee discretion. <i>N.

In [6]:
files = ! ls {load_folder}/*.yml
files

['lvl2/Acid_Arrow.yml',
 'lvl2/Aid.yml',
 'lvl2/Augury.yml',
 'lvl2/Barkskin.yml',
 'lvl2/Blur.yml',
 'lvl2/Captivate.yml',
 'lvl2/Cause_Blindness.yml',
 'lvl2/Cause_Deafness.yml',
 'lvl2/Chameleon.yml',
 'lvl2/Charm_Person_or_Beast.yml',
 'lvl2/Continuous_Light.yml',
 'lvl2/Cool_Metal.yml',
 'lvl2/Cudgel_of_Bone.yml',
 'lvl2/Cure_Light_Burns.yml',
 'lvl2/Cure_Moderate_Wounds.yml',
 'lvl2/Danse_Macabre.yml',
 'lvl2/Delay_Poison.yml',
 'lvl2/Detect_Body_Heat.yml',
 'lvl2/Detect_Neutrality.yml',
 'lvl2/Distinguish_Alignment.yml',
 'lvl2/Enthral.yml',
 'lvl2/Extra-dimensional_Pocket.yml',
 'lvl2/Extrasensory_Perception.yml',
 'lvl2/Find_Plants.yml',
 'lvl2/Find_Traps.yml',
 'lvl2/Fire_Trap.yml',
 'lvl2/Fire_Web.yml',
 'lvl2/Flame_Blade.yml',
 'lvl2/Flaming_Sphere.yml',
 'lvl2/Fog_Cloud.yml',
 'lvl2/Fools_Gold.yml',
 'lvl2/Forest_of_Bones.yml',
 'lvl2/Frost_Sphere.yml',
 'lvl2/Ghoul_Touch.yml',
 'lvl2/Glitterdust.yml',
 'lvl2/Gust_of_Wind.yml',
 'lvl2/Heat_Metal.yml',
 'lvl2/Hold_Person.ym

In [7]:
def load_yml_spell(file_name):
    with open(file_name, "r") as f:
        spell = yaml.safe_load(f)
    return spell

In [8]:
def spell_preload_check(spell):
    with db_cur() as cur:
        cur.execute(
            """
            SELECT *
            FROM spells
            WHERE spell_id = ?;
            """,
            (spell["id"],),
        )
        db_spell = cur.fetchone()
    assert spell["name"] == db_spell["spell_name"]
    assert isinstance(spell["id"], int)
    assert isinstance(spell["pp"], int)
    assert isinstance(spell["Rng"], (str, int))
    assert isinstance(spell["Dur"], str)
    assert spell["Rev"] in [0, 1]
    assert isinstance(spell["Desc"], str)
    assert spell["Desc"][:3] == "<p>"
    if spell["id"] not in [53]:
        assert spell["Desc"][-4:] == "</p>"



In [9]:
def update_spell(spell):
    with db_cur() as cur:
        cur.execute(
            """
            UPDATE spells
            SET rng = ?
                , dur = ?
                , reversible = ?
                , pp = ?
                , spell_desc = ?
            WHERE spell_id = ?;
            """,
            (
                spell["Rng"],
                spell["Dur"],
                spell["Rev"],
                spell["pp"],
                spell["Desc"],
                spell["id"],
            ),
        )
    return 

### Test parsing all the files

In [13]:
for f in files:
    print(f"checking {f}")
    spell = load_yml_spell(f)
    spell_preload_check(spell)
    print("success")
    print()

checking lvl2/Acid_Arrow.yml
success

checking lvl2/Aid.yml
success

checking lvl2/Augury.yml
success

checking lvl2/Barkskin.yml
success

checking lvl2/Blur.yml
success

checking lvl2/Captivate.yml
success

checking lvl2/Cause_Blindness.yml
success

checking lvl2/Cause_Deafness.yml
success

checking lvl2/Chameleon.yml
success

checking lvl2/Charm_Person_or_Beast.yml
success

checking lvl2/Continuous_Light.yml
success

checking lvl2/Cool_Metal.yml
success

checking lvl2/Cudgel_of_Bone.yml
success

checking lvl2/Cure_Light_Burns.yml
success

checking lvl2/Cure_Moderate_Wounds.yml
success

checking lvl2/Danse_Macabre.yml
success

checking lvl2/Delay_Poison.yml
success

checking lvl2/Detect_Body_Heat.yml
success

checking lvl2/Detect_Neutrality.yml
success

checking lvl2/Distinguish_Alignment.yml
success

checking lvl2/Enthral.yml
success

checking lvl2/Extra-dimensional_Pocket.yml
success

checking lvl2/Extrasensory_Perception.yml
success

checking lvl2/Find_Plants.yml
success

checking 

Do it for real

In [14]:
for f in files:
    spell = load_yml_spell(f)
    spell_preload_check(spell)
    update_spell(spell)

In [15]:
load_yml_spell(files[-1])

{'id': 455,
 'name': 'Zombie Visage',
 'pp': 247,
 'Lvl': ['ill 3', 'nec 2'],
 'Rng': 0,
 'Dur': '1 round per CA level',
 'Rev': 0,
 'Desc': '<p>The sorcerer’s face and hands become like those of a <b>zombie</b>, accompanied by a conspicuous foetor and an aura of undeath, allowing one to pass amongst unintelligent undead without notice. This mien is merely a necromantic illusion, however. Ordinary animals that glimpse the sorcerer must make morale checks at −2 or flee in terror for 1d4 rounds. <i>Zombie visage</i> is broken by <i>dispel magic</i> or <i>dispel phantasm</i>; a <i>turn undead</i> attempt versus <i>Undead Type 2</i>, if successful, will cause the face to crumble and fall away from the sorcerer.</p>'}